In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Fonctions utiles

In [13]:
def f_html(lien):
    return requests.get(lien).text.encode("utf-8")

def f_soup(lien):
    return BeautifulSoup(f_html(lien),'lxml')

def f_convertdict(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

def delete_several_cha_from_end(string,n):
    for i in range(n):
        string = string.rstrip(string[-1])
    return string

def delete_byindex(string,index):
    if len(string) > index:
        return string[0 : index : ] + string[index + 1 : :]



Récupérer nom et lien des rayons

In [14]:
def search_rayons(lien,list_rayons_principal=[]):
    acceuil = f_soup(lien)
    rayon_tag = acceuil.find_all('div',class_='navigation-node trigger-sublevel not-loaded')

    for rayon in rayon_tag:

        #add nom in list
        rayon_principal = rayon.text.strip().replace(' ','').replace(",","")

        #add lien in list
        rayon_lien= rayon.a['href']
        rayon_lien =f'{lien}{rayon_lien}'

        dict_rayon_principal={
            'rayon principal' : rayon_principal,
            'lien rayon' : rayon_lien 
        }
        list_rayons_principal.append(dict_rayon_principal)

    return list_rayons_principal

Récupérer noms des sous rayons

In [15]:
def search_sous_rayon (lien,lienp ='https://www.auchan.fr',list_sous_rayon=[]):
    soup = f_soup(lien)
    rayon_sous_tag = soup.find_all('div', class_='universe-block')
    for rayon_sous in rayon_sous_tag:

        # add name of sub rayon
        nom_rayon_sous = rayon_sous.find('div', class_='universe-block__title')
        nom_rayon_sous = nom_rayon_sous.text.strip().replace(' ','').replace(',','')
        
        #add link of sub rayon
        lien_sous_rayon = rayon_sous.a['href']
        lien_sous_rayon =f'{lienp}{lien_sous_rayon}'
        
        dict_rayon_sous={
            'sous rayon' : nom_rayon_sous,
            'lien sous rayon' : lien_sous_rayon
        }
        
        list_sous_rayon.append(dict_rayon_sous)

    return list_sous_rayon

Assemble list rayon principal et sous rayon

In [16]:
def assemble_rayonp_rayons(lien):
    #initialize dataframe 
    df_all_rayon = pd.DataFrame(columns = ['rayon principal', 'lien rayon', 'sous rayon', 'lien sous rayon'])
 
    #search rayons principal
    df_rayon_principal = pd.DataFrame(search_rayons(lien))

    for i,lien_rayon in enumerate(df_rayon_principal['lien rayon']):

        list_rayon_sous = search_sous_rayon(lien_rayon)

        if len(list_rayon_sous) == 0:
            list_rayon = [{
                'rayon principal' : df_rayon_principal.iloc[i]['rayon principal'],
                'lien rayon' : df_rayon_principal.iloc[i]['lien rayon'],
                'sous rayon' : 'N/A',
                'lien sous rayon' :'N/A'}]
            df_rayon = pd.DataFrame(list_rayon)
            df_all_rayon = pd.concat([df_all_rayon,df_rayon],ignore_index=True)


        else : 
            for k,sous_rayon in enumerate(list_rayon_sous):

                df_sous_rayon = pd.DataFrame([sous_rayon])
    
                list_rayon = [{
                    'rayon principal' : df_rayon_principal.iloc[i]['rayon principal'],
                    'lien rayon' : df_rayon_principal.iloc[i]['lien rayon'],
                    'sous rayon' : df_sous_rayon.iloc[0]['sous rayon'],
                    'lien sous rayon' : df_sous_rayon.iloc[0]['lien sous rayon']}]
                df_rayon = pd.DataFrame(list_rayon)
                df_all_rayon = pd.concat([df_all_rayon,df_rayon],ignore_index=True)
                
        list_rayon_sous.clear()

    return df_all_rayon

lien ='https://www.auchan.fr'
df_all_rayon = assemble_rayonp_rayons(lien)
#df_all_rayon.to_csv("list_all_rayon.csv")
print(df_all_rayon)



            rayon principal  \
0    Jouetsjeuxvidéoloisirs   
1             AucoeurdeNoël   
2             AucoeurdeNoël   
3             AucoeurdeNoël   
4             AucoeurdeNoël   
..                      ...   
73    Electroménagercuisine   
74  Meubledécolingedemaison   
75      Modebijouxbagagerie   
76               Nosmarques   
77     SAVservicesetvoyages   

                                           lien rayon  \
0   https://www.auchan.fr/jouets-jeux-video-loisir...   
1   https://www.auchan.fr/au-coeur-de-noel/ca-b202...   
2   https://www.auchan.fr/au-coeur-de-noel/ca-b202...   
3   https://www.auchan.fr/au-coeur-de-noel/ca-b202...   
4   https://www.auchan.fr/au-coeur-de-noel/ca-b202...   
..                                                ...   
73  https://www.auchan.fr/electromenager-cuisine/c...   
74  https://www.auchan.fr/meuble-deco-linge-de-mai...   
75  https://www.auchan.fr/mode-bijoux-bagagerie/ca...   
76  https://www.auchan.fr/produits-auchan/ca-b2022...   


Récupérer produits

In [17]:

acceuil = f_soup(lien_srayon)
produits = acceuil.find_all('div',class_="product-thumbnail__details")
#print(produits)

produit_list =[]
for produit in produits:

    produit = produit.text.replace(' ','').replace('\n',',')
    produit_list.append(produit)
    #print(produit)

print(produit_list)


NameError: name 'lien_srayon' is not defined